In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


Installation of all libraries.

In [2]:
import numpy as np # library to handle data in a vectorized manner
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim
# convert an address into latitude and longitude values

import requests # library to handle requests


# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
import folium 
# map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


Extracted the data from Wikipedia and coverrted the data in tabular format.

In [3]:

data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text
# Parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# Create a list to store neighbourhood data
neighborhoodList = []
# Append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
# Create a new DataFrame from the list
Neighbor_df = pd.DataFrame({"Neighborhood": neighborhoodList})
Neighbor_df.head()

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"


In [4]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


Import Geocoder package to get the latitude and lonngitude.

In [5]:
import geocoder
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bengaluru, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Neighbor_df["Neighborhood"].tolist()]

In [6]:
Latitude = 0
Longitude = 0
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
Neighbor_df['Latitude'] = df_coords['Latitude']
Neighbor_df['Longitude'] = df_coords['Longitude']
print(Neighbor_df.shape)
Neighbor_df.head()

(139, 3)


,Neighborhood,Latitude,Longitude
0,List of areas in Bangalore Cantonment,12.97568,77.60539
1,List of areas in Bengaluru Pete,12.96053,77.64385
2,List of neighbourhoods in Bangalore,12.88813,77.57735
3,Adugodi,12.94402,77.60800
4,"Agara, Bangalore",12.84283,77.48759


In [7]:
address = 'Bengaluru, India'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru, India 12.9791198, 77.5912997.


Folium Map for Bangalore Neibothood.

In [9]:
map_Nei = folium.Map(location=[latitude, longitude], zoom_start=11)
# Adding markers to map
for lat, lng, neighborhood in zip(Neighbor_df['Latitude'],  Neighbor_df['Longitude'], Neighbor_df['Neighborhood']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_Nei)
map_Nei

Foursquare API to get the venues

In [10]:
from pandas.io.json import json_normalize 

In [11]:
CLIENT_ID = '5MMMYIK4HVXHAEV5UI5BCM20IJBUPJW4JGSLQKFB1ODSPBDT' # your Foursquare ID
CLIENT_SECRET = 'DFZAXYKD42ATIVFFZHMYOGPLLRO0O5WVJC4CIQGFJYQ0LF2X' # your Foursquare Secret
VERSION = '20180604'
radius = 3000
LIMIT = 100
venues = []
for lat, long, neighborhood in zip(Neighbor_df['Latitude'], Neighbor_df['Longitude'], Neighbor_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,LIMIT)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((neighborhood,lat,long,venue['venue']['name'],venue['venue']['location']['lat'],venue['venue']['location']    ['lng'],venue['venue']['categories'][0]['name']))  
        
    
# Create the API request URL

# Make the GET request

# Return only relevant information for each nearby venue
      
    


# Make the GET request

# Create the API request URL
#url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,LIMIT)

# Return only relevant information for each nearby venue
    
   

Dataframe with the venues returned by Foursquare API.

In [12]:
venues_df = pd.DataFrame(venues)
# Defining the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.shape


(9584, 7)


(9584, 7)

In [13]:
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 209 unique categories.


In [14]:
venues_df['VenueCategory'].unique()[:50]

array(['Lounge', 'Bookstore', 'Pizza Place', 'Plaza', 'Toy / Game Store',
       'American Restaurant', 'Café', 'Coffee Shop', 'Pub',
       'Burger Joint', 'Ice Cream Shop', 'Cricket Ground',
       'Indian Restaurant', 'Breakfast Spot', 'Deli / Bodega', 'Brewery',
       'Afghan Restaurant', 'Andhra Restaurant', 'Sushi Restaurant',
       'Park', 'Shopping Mall', 'Italian Restaurant',
       'Japanese Restaurant', 'Hotel', 'French Restaurant',
       'Bubble Tea Shop', 'Gym', 'Cocktail Bar', 'Furniture / Home Store',
       'Cupcake Shop', 'Soccer Stadium', 'Tea Room', 'Department Store',
       'Athletics & Sports', 'Gym / Fitness Center', 'Asian Restaurant',
       'Bakery', 'Chinese Restaurant', 'Clothing Store', 'Dessert Shop',
       "Women's Store", 'Road', 'Fried Chicken Joint', 'Market',
       'Korean Restaurant', 'Eastern European Restaurant', 'Steakhouse',
       'Bed & Breakfast', 'Thai Restaurant', 'Restaurant'], dtype=object)

Total Neighborhoods with Venues and latitude, longitudes

In [15]:
venues_df = pd.DataFrame(venues)
# Defining the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(9584, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of areas in Bangalore Cantonment,12.97568,77.60539,The 13th Floor,12.975364,77.604995,Lounge
1,List of areas in Bangalore Cantonment,12.97568,77.60539,Blossom Book House,12.975042,77.604813,Bookstore
2,List of areas in Bangalore Cantonment,12.97568,77.60539,Brik Oven,12.974757,77.605441,Pizza Place
3,List of areas in Bangalore Cantonment,12.97568,77.60539,M.G Road Boulevard,12.975771,77.603979,Plaza
4,List of areas in Bangalore Cantonment,12.97568,77.60539,The Entertainment Store,12.975413,77.603045,Toy / Game Store


In [16]:
Neigh_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
# Adding neighborhood column back to dataframe
Neigh_onehot['Neighborhoods'] = venues_df['Neighborhood']
# Moving neighbourhood column to the first column
fixed_columns = [Neigh_onehot.columns[-1]] + list(Neigh_onehot.columns[:-1])
Neigh_onehot = Neigh_onehot[fixed_columns]
print(Neigh_onehot.shape)

(9584, 210)


In [17]:
Neigh_grouped=Neigh_onehot.groupby(["Neighborhoods"]).sum().reset_index()
print(Neigh_grouped.shape)
Neigh_grouped.head(10)

(139, 210)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,2,1,0,1,0,0,0,1,0,0,1,1,0,2,3,0,0,1,0,0,0,0,0,0,0,6,1,0,0,3,0,2,1,2,0,0,0,1,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,1,0,0,0,1,0,4,0,0,2,0,12,1,1,0,0,2,0,1,2,0,0,0,1,0,0,0,4,0,0,0,1,0,0,0,2,0,1,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,0,0,0,0,1,0,0,0,1,0,0,1,2,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,1
1,"Agara, Bangalore",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Ananthnagar,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,2,0,0,0,0,4,

Number of Multiplexes from the explored Dataset.

In [18]:
print('Number of Multiplexes:',len((Neigh_grouped[Neigh_grouped["Multiplex"] > 0])))

Number of Multiplexes: 59


In [19]:
Neigh_mult = Neigh_grouped[["Neighborhoods","Multiplex"]]

In [20]:
kclusters = 3
Neigh_clustering = Neigh_mult.drop(["Neighborhoods"], 1)
# Run k-means clustering algorithm
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(Neigh_clustering)
# Checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



array([2, 1, 2, 1, 0, 1, 0, 1, 1, 2])

In [21]:
k_means_cluster_centers = kmeans.cluster_centers_
k_means_cluster_centers

array([[ 3.27272727e+00],
       [-3.33066907e-16],
       [ 1.24324324e+00]])

In [22]:
Neigh_merged = Neigh_mult.copy()
# Add the clustering labels
Neigh_merged["Cluster Labels"] = kmeans.labels_
Neigh_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Neigh_merged.head(10)

,Neighborhood,Multiplex,Cluster Labels
0,Adugodi,2,2
1,"Agara, Bangalore",0,1
2,Ananthnagar,2,2
3,Anjanapura,0,1
4,Arekere,4,0
5,Austin Town,0,1
6,BTM Layout,4,0
7,Babusapalya,0,1
8,"Bagalur, Bangalore Urban",0,1
9,Bahubalinagar,1,2


Cluster Labels with Neighborhoods and multiplexes. Added Longitude and Latitude of the nighborhoods.

In [26]:
Neigh_merged['Latitude'] = Neighbor_df['Latitude']
Neigh_merged['Longitude'] = Neighbor_df['Longitude']
# Sorting the results by Cluster Labels
Neigh_merged.sort_values(["Cluster Labels"], inplace=True)
Neigh_merged.head(10)

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
138,Yeswanthpur,3,0,13.029540,77.540220
4,Arekere,4,0,12.842830,77.487590
13,Basaveshwaranagar,4,0,13.019647,77.654694
123,Taverekere,3,0,12.922450,77.610330
128,Varthur,3,0,12.943480,77.747030
32,Devarachikkanahalli,4,0,12.978999,77.656132
17,Bilekahalli,3,0,12.966180,77.586900
116,"Siddapura, Bangalore",3,0,12.956130,77.731960
103,Rajajinagar,4,0,13.005440,77.556930
101,Puttenahalli,3,0,12.894340,77.585070


Map to show the clusters of Neighborhoods.

In [31]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Neigh_merged['Latitude'], Neigh_merged['Longitude'], Neigh_merged['Neighborhood'], Neigh_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)

    
map_clusters

Number of Multiplexes in the clusters.

In [35]:
print(len(Neigh_merged.loc[Neigh_merged['Cluster Labels'] == 0]))
print( len(Neigh_merged.loc[Neigh_merged['Cluster Labels'] == 1]))
print(len(Neigh_merged.loc[Neigh_merged['Cluster Labels'] == 2]))
print(len(Neigh_merged.loc[Neigh_merged['Cluster Labels'] == 3]))
print(len(Neigh_merged.loc[Neigh_merged['Cluster Labels'] == 4]))


22
80
37
0
0


In [36]:
Neigh_merged.groupby('Multiplex')['Cluster Labels'].value_counts()

Multiplex  Cluster Labels
0          1                 80
1          2                 28
2          2                  9
3          0                 16
4          0                  6
Name: Cluster Labels, dtype: int64